#### Ingestion del archivo movie_languages

In [0]:
dbutils.widgets.text("p_enviroment","")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

##### Paso 1 - Leer el archivo Json usando "DataFrameReader" en Spark

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType

In [0]:
movie_languages_schema = StructType(fields = [
  StructField("movieId", IntegerType(), False),
  StructField("languageId", IntegerType(),False),
  StructField("languageRoleId",IntegerType(), False)
])

In [0]:
movie_language_df = spark.read.option("multiline","True").schema(movie_languages_schema) \
                                                        .json(f"{bronze_folder_path}/{v_file_date}/movie_language")

#### Paso 2 - Agregar los campos faltnates y cambiar de nombre
##### 1. Cambiar el nombre de "movieId" a "movie_id"
##### 2. Cambiar el nombre de "languageId" a "language_id"
##### 3. Agregar "enviroment" y Agregar "ingestion_date"

In [0]:
from pyspark.sql.functions import lit, current_timestamp,col

In [0]:
movie_language_column_df = add_ingestion_date(movie_language_df).withColumnRenamed("movieId","movie_id") \
                                .withColumnRenamed("languageId","language_id") \
                                .withColumn("enviromen",lit(v_enviroment)) \
                                .withColumn("file_date",lit(v_file_date))

#### Paso 3 - Eliminar los campos innecesarios

In [0]:
movie_language_final = movie_language_column_df.drop(col("languageRoleId"))

#### Paso 4 - Guardar el DF en parquet

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.language_id = src.language_id AND tgt.file_date = src.file_date'
merge_delta_lake(movie_language_final, "movie_silver", "movie_languages", silver_foler_path, merge_condition, "file_date")

#overwrite_partition(movie_language_final,"movie_silver","movie_languages","file_date")

In [0]:
%sql
SELECT COUNT(1), file_date
FROM movie_silver.movie_languages
GROUP BY file_date

In [0]:
    dbutils.notebook.exit("Success!")